# BentoML Scikit-learn Tutorial


This is a sample project demonstrating basic usage of [BentoML](https://github.com/bentoml) with
Scikit-learn.

In this project, we will train a classifier model using Scikit-learn and the Iris dataset, build
an prediction service for serving the trained model via an HTTP server, and containerize the 
model server as a docker image for production deployment.


Link to source code: https://github.com/bentoml/gallery/tree/main/quickstart

### Install Dependencies

Install required python packages:

In [1]:
#!pip install -r https://raw.githubusercontent.com/bentoml/gallery/main/quickstart/requirements.txt   --user
!pip install scikit-learn
!pip install pandas
!pip install -U bentoml  

##  Model Training

In [2]:
from sklearn import svm, datasets

# Load training data
iris = datasets.load_iris()
X, y = iris.data, iris.target

# Model Training
clf = svm.SVC()
clf.fit(X, y)

SVC()

Save the `clf` model instance to BentoML local model store:

In [3]:
import bentoml
bentoml.sklearn.save_model("iris_clf", clf)

Model(tag="iris_clf:d57xpvqau6t6x3c4", path="C:\Users\hwang\bentoml\models\iris_clf\d57xpvqau6t6x3c4\")

Models saved can be accessed via `bentoml models` CLI command:

In [4]:
!bentoml models get iris_clf:latest

name: iris_clf                                                                 
version: d57xpvqau6t6x3c4                                                      
module: bentoml.sklearn                                                        
labels: {}                                                                     
options: {}                                                                    
metadata: {}                                                                   
context:                                                                       
  framework_name: sklearn                                                      
  framework_versions:                                                          
    scikit-learn: 1.1.1                                                        
  bentoml_version: 1.0.0rc3                                                    
  python_version: 3.9.7                                                        
signatures:                             

In [1]:
!bentoml models list

 Tag                    Module                 Size        Creation Time       
 spam_detector:7636ah���  bentoml.picklable_mo���  426.38 KiB  2022-07-10 16:29:29 
 spam_detector:cwz2hz���  bentoml.picklable_mo���  426.38 KiB  2022-07-10 03:08:22 
 mypicklablemodel:2wt���  bentoml.picklable_mo���  426.38 KiB  2022-07-10 01:26:22 
 mypicklablemodel:oft���  bentoml.picklable_mo���  426.38 KiB  2022-07-10 01:23:34 
 iiris_clf:as5goziacc���  bentoml.picklable_mo���  426.37 KiB  2022-07-10 01:20:31 
 tensorflow_mnist:qmn���  bentoml.tensorflow     10.63 MiB   2022-07-09 02:36:49 
 tensorflow_mnist:ols���  bentoml.tensorflow     10.63 MiB   2022-07-09 02:36:22 
 agaricus:ku24bnx7jsb���  bentoml.xgboost        2.73 KiB    2022-07-09 01:59:45 
 agaricus:lz5x4d77jkl���  bentoml.xgboost        2.73 KiB    2022-07-09 01:45:41 
 iris_clf:5fjpqlp7isa���  bentoml.sklearn        5.77 KiB    2022-07-09 01:06:37 
 iris_clf:gnq6orh7iom���  bentoml.sklearn        5.77 KiB    2022-07-09 00:54:23 
 iris_cl

To verify that the saved model can be loaded correctly:

In [5]:
loaded_model = bentoml.sklearn.load_model("iris_clf:latest")

loaded_model.predict([[5.9, 3. , 5.1, 1.8]])

array([2])

In BentoML, the recommended way of running ML model inference in serving is via Runner, which 
gives BentoML more flexibility in terms of how to schedule the inference computation, how to 
batch inference requests and take advantage of hardware resoureces available. Saved models can
be loaded as Runner instance as shown below:


In [6]:
# Create a Runner instance:
iris_clf_runner = bentoml.sklearn.get("iris_clf:latest").to_runner()

# Runner#init_local initializes the model in current process, this is meant for development and testing only:
iris_clf_runner.init_local()

# This should yield the same result as the loaded model:
iris_clf_runner.predict.run([[5.9, 3., 5.1, 1.8]])

'Runner.init_local' is for debugging and testing only


array([2])

## Serving the model

A simple BentoML Service that serves the model saved above look like this:

In [7]:
%%writefile service.py
import numpy as np
import bentoml
from bentoml.io import NumpyNdarray

iris_clf_runner = bentoml.sklearn.get("iris_clf:latest").to_runner()

svc = bentoml.Service("iris_classifier", runners=[iris_clf_runner])

@svc.api(input=NumpyNdarray(), output=NumpyNdarray())
def classify(input_series: np.ndarray) -> np.ndarray:
    return iris_clf_runner.predict.run(input_series)


Overwriting service.py


# Iris Classifier (난초 구분) 설명

https://www.youtube.com/watch?v=pTjsr_0YWas

1. Sepal length (cm)
2. Sepal width (cm)
3. Petal length (cm)
4. petal width (cm)
5. class --- 3 classes  Iris Setosa, Iris Versicolour, Iris Virginica

예. 
## [4.9, 3 , 1.4, 0.2] --> 0 (Iris Setosa)
## [6.1, 2.9, 4.7, 1.4] --> 1 (Iris Versicolour)
## [5.8, 2.7, 5.1, 1.9] --> 2 (Iris Virginica)
## [6.2, 3.4, 5.4, 2.3] --> 2 (Iris Virginica)

In [1]:
#print (iris)

Note: using `%%writefile` here because `bentoml.Service` definition must be created in its own `.py` file

Start a dev model server to test out the service defined above:

# 서빙
Open your web browser at http://127.0.0.1:3000 to view the Bento UI for sending test requests.
Windows OS에서는 --reload option 없이 돌려야 한다.

In [ ]:
#!bentoml serve service.py:svc --reload
!bentoml serve service.py:svc
    
# 참고로, AWS에서는 host 를 써준다. 여기서 Host는 AWS의 EC2의 프라이빗 IPv4 주소
#!bentoml serve nb_service.py:svc --host 172.31.90.78


웹 브라우저 사용해서 확인할 수 있다. http://127.0.0.1:3000 (AWS EC2를 외부 브라우저로 접속할 경우, 퍼블릭 IPv4 주소를 127.0.0.1 대신 사용한다)로 연결해서 녹색으로 된 app POST 섹션을 클릭한다. "Try it out" 클릭한 후에 값 부분을 원하는 [5.9, 3, 5.1, 1.8] 으로 넣고, execute 를 하면 된다.

AWS EC2 에서 serving 할 때, 이를 외부에서 확인하기 위해서는, 3000 번 port 를 보안탭에서 inbound 규칙에 추가한다. 포트 범위: 3000 프로토콜: TCP 원본: 0.0.0.0/0

You may also send request with `curl` command or any HTTP client, e.g.:

```bash
curl -X POST -H "content-type: application/json" --data "[[5.9, 3, 5.1, 1.8]]" http://127.0.0.1:3000/classify
```


### Build Bento for deployment

Bento is the distribution format in BentoML which captures all the source code, model files, config
files and dependency specifications required for running the service for production deployment. Think 
of it as Docker/Container designed for machine learning models.

To begin with building Bento, create a `bentofile.yaml` under your project directory:

In [8]:
%%writefile bentofile.yaml
service: "service.py:svc"
labels:
  owner: bentoml-team
  project: gallery
include:
- "*.py"
python:
  packages:
    - scikit-learn
    - pandas

Overwriting bentofile.yaml


Next, run `bentoml build` from current directory to start the Bento build:

In [9]:
!bentoml build

Building BentoML service "iris_classifier:hp2yqraau6ghb3c4" from build context "C:\Users\hwang\OneDrive\Documents\GitHub\MLOpsToyProject\notebooks"
Packing model "iris_clf:d57xpvqau6t6x3c4"
Locking PyPI package versions..

�뻽�뻽�뻽�뻽�뻽�뻽�븮�뼇�뻽�뻽�뻽�뻽�뻽�뻽�뻽�븮�뻽�뻽�뻽�븮�뼇�뼇�뻽�뻽�븮�뻽�뻽�뻽�뻽�뻽�뻽�뻽�뻽�븮�뼇�뻽�뻽�뻽�뻽�뻽�븮�뼇�뻽�뻽�뻽�븮�뼇�뼇�뼇�뻽�뻽�뻽�븮�뻽�뻽�븮�뼇�뼇�뼇�뼇�뼇
�뻽�뻽�븫�븧�븧�뻽�뻽�븮�뻽�뻽�븫�븧�븧�븧�븧�븴�뻽�뻽�뻽�뻽�븮�뼇�뻽�뻽�븨�븱�븧�븧�뻽�뻽�븫�븧�븧�븴�뻽�뻽�븫�븧�븧�뻽�뻽�븮�뻽�뻽�뻽�뻽�븮�뼇�뻽�뻽�뻽�뻽�븨�뻽�뻽�븨�뼇�뼇�뼇�뼇�뼇
�뻽�뻽�뻽�뻽�뻽�뻽�븽�븴�뻽�뻽�뻽�뻽�뻽�븮�뼇�뼇�뻽�뻽�븫�뻽�뻽�븮�뻽�뻽�븨�뼇�뼇�뼇�뻽�뻽�븨�뼇�뼇�뼇�뻽�뻽�븨�뼇�뼇�뻽�뻽�븨�뻽�뻽�븫�뻽�뻽�뻽�뻽�븫�뻽�뻽�븨�뻽�뻽�븨�뼇�뼇�뼇�뼇�뼇
�뻽�뻽�븫�븧�븧�뻽�뻽�븮�뻽�뻽�븫�븧�븧�븴�뼇�뼇�뻽�뻽�븨�븱�뻽�뻽�뻽�뻽�븨�뼇�뼇�뼇�뻽�뻽�븨�뼇�뼇�뼇�뻽�뻽�븨�뼇�뼇�뻽�뻽�븨�뻽�뻽�븨�븱�뻽�뻽�븫�븴�뻽�뻽�븨�뻽�뻽�븨�뼇�뼇�뼇�뼇�뼇
�뻽�뻽�뻽�뻽�뻽�뻽�븽�븴�뻽�뻽�뻽�뻽�뻽�뻽�뻽�븮�뻽�뻽�븨�뼇�븱�뻽�뻽�뻽�븨�뼇�뼇�뼇�뻽�뻽�븨�뼇�뼇�뼇�븱�뻽�뻽�뻽�뻽�뻽�븫�븴�뻽�뻽�븨�뼇�븱�븧�븴�뼇�뻽�뻽�븨�뻽�뻽�뻽�뻽�뻽�뻽�뻽�븮
�븱�븧�븧�븧�븧�븧�븴�뼇�븱�븧�븧�븧�븧�븧�븧�븴�븱�븧�븴�뼇�뼇�븱�븧�븧�븴�뼇�뼇�뼇�븱�븧�븴�뼇�뼇�뼇�뼇�븱�븧�븧�븧�븧�븴�뼇�븱�븧�븴�뼇�뼇�뼇�뼇�뼇�븱�븧�븴�븱�븧�븧�븧�븧�븧�븧�븴

Successfully built Bento(tag="iris_cla

In [10]:
!bentoml containerize iris_classifier:latest

Building docker image for Bento(tag="iris_classifier:hp2yqraau6ghb3c4")...
Successfully built docker image "iris_classifier:hp2yqraau6ghb3c4"


#1 [internal] load build definition from Dockerfile
#1 transferring dockerfile: 2.19kB done
#1 DONE 0.1s

#2 [internal] load .dockerignore
#2 transferring context: 2B done
#2 DONE 0.1s

#3 resolve image config for docker.io/docker/dockerfile:1.4-labs
#3 DONE 0.6s

#4 docker-image://docker.io/docker/dockerfile:1.4-labs@sha256:b50ad4af81d1c76ab7c0e1ffc216909e7adc23e99910243e1c88331c2a8ef52d
#4 CACHED

#5 [internal] load build definition from Dockerfile
#5 DONE 0.0s

#6 [internal] load .dockerignore
#6 DONE 0.0s

#7 [internal] load metadata for docker.io/library/python:3.9-slim
#7 DONE 0.4s

#8 [cached 1/1] FROM docker.io/library/python:3.9-slim@sha256:c01a2db78654c1923da84aa41b829f6162011e3a75db255c24ea16fa2ad563a0
#8 DONE 0.0s

#9 [internal] load build context
#9 transferring context: 15.86kB done
#9 DONE 0.1s

#10 [stage-1  3/11] RUN --mount=type=cache,from=cached,sharing=shared,target=/var/cache/apt     --mount=type=cache,from=cached,sharing=shared,target=/var/lib/apt 	apt-get update 

A new Bento is now built and saved to local Bento store. You can view and manage it via 
`bentoml list`,`bentoml get` and `bentoml delete` CLI command.

## Containerize and Deployment

In [ ]:
!bentoml containerize iris_classifier:latest

In [ ]:
!docker run -p 3000:3000 iris_classifier:invwzzsw7li6zckb2ie5eubhd 

Bento is designed to be deployed to run efficiently in a variety of different environments.
And there are lots of deployment options and tools as part of the BentoML eco-system, such as 
[Yatai](https://github.com/bentoml/Yatai) and [bentoctl](https://github.com/bentoml/bentoctl) for
direct deployment to cloud platforms.

In this guide, we will show you the most basic way of deploying a Bento, which is converting a Bento
into a Docker image containing the HTTP model server.

Make sure you have docker installed and docker deamon running, and run the following commnand:

```bash
bentoml containerize iris_classifier:latest
```

This will build a new docker image with all source code, model files and dependencies in place,
and ready for production deployment. To start a container with this docker image locally, run:

```bash
docker run -p 3000:3000 iris_classifier:invwzzsw7li6zckb2ie5eubhd 
```

## What's Next?

- 👉 [Pop into our Slack community!](https://l.linklyhq.com/l/ktO8) We're happy to help with any issue you face or even just to meet you and hear what you're working on.

- Dive deeper into the [Core Concepts](https://docs.bentoml.org/en/latest/concepts/index.html) in BentoML
- Learn how to use BentoML with other ML Frameworks at [Frameworks Guide](https://docs.bentoml.org/en/latest/frameworks/index.html) or check out other [gallery projects](https://github.com/bentoml/gallery)
- Learn more about model deployment options for Bento:
  - [🦄️ Yatai](https://github.com/bentoml/Yatai): Model Deployment at scale on Kubernetes
  - [🚀 bentoctl](https://github.com/bentoml/bentoctl): Fast model deployment on any cloud platform
